In [1]:
import pandas as pd
import numpy as np
import gc
import time
from contextlib import contextmanager
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import roc_auc_score, roc_curve, f1_score
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pickle
import os
from sklearn.svm import SVC
warnings.simplefilter(action='ignore', category=FutureWarning)
dataroot = "../data/"
cacheRoot = "../cache/"
subRoot = "../submission/"

In [3]:
# import os

# def list_files(startpath):
#     for root, dirs, files in os.walk(startpath):
#         level = root.replace(startpath, '').count(os.sep)
#         indent = ' ' * 4 * (level)
#         print('{}{}/'.format(indent, os.path.basename(root)))
#         subindent = ' ' * 4 * (level + 1)
#         for f in files:
#             print('{}{}'.format(subindent, f))
# list_files("../")
def get_embeding(fname, embname):
    f = open(fname)
    embeding_lines = f.readlines()
    f.close()
    mapfunc = lambda x: list( map( float, x ) )
    embeding_lines = [li.replace("\n","").split(" ") for li in embeding_lines[1:]]
    embeding_lines = [ [ int(line[0]) ] +  mapfunc( line[1:]  )   for line in   embeding_lines ]
    cols = ["UID"] + [ embname  + str(i) for i in range( len(embeding_lines[0]) -1 )]
    embeding_df = pd.DataFrame(embeding_lines, columns=cols )
    del embeding_lines
    return embeding_df



In [1]:
data =  pd.read_csv(cacheRoot + "grouping_features.csv")

NameError: name 'pd' is not defined

['device1_edglist_emb_dim36.emb',
 'device_code2_edglist_emb_dim36.emb',
 'geo_code_edglist.embeddings',
 'geo_code_edglist_emb_dim36.emb',
 'ip1_edglist.embeddings',
 'ip1_edglist_emb_dim36.emb',
 'mac1_edglist_emb_dim36.emb',
 'mac1_edglist_noweit_.embeddings',
 'merchant_edglist_emb_dim36.emb',
 'merchant_weighted_edglist_DeepWalk.embeddings',
 'wifi_edglist.embeddings',
 'wifi_edglist_emb_dim36.emb']

In [103]:
# mac1_emb = get_embeding(cacheRoot + "mac1_.emb", "mac1_emb_")
# merchant_dbk = get_embeding(cacheRoot + "merchant_weighted_edglist_DeepWalk.embeddings", "merchant_deepwalk_")
# mac1_emb = mac1_emb[ mac1_emb.UID.map(lambda x: x<= 131587) ] 
# merchant_dbk = merchant_dbk[ merchant_dbk.UID.map(lambda x: x<= 131587) ] 


# merchant_emb = get_embeding(cacheRoot + "merchant_.emb", "merchant_emb_")
# merchant_emb = merchant_emb[ merchant_emb.UID.map(lambda x: x<= 131587) ] 
# mac_merch = pd.DataFrame()
# mac_merch["UID"] = merchant_dbk.UID.tolist() +  mac1_emb.UID.tolist()
# mac_merch = mac_merch.merge( mac1_emb, on = ["UID"],how = "left"  )
# mac_merch = mac_merch.merge( merchant_dbk, on = ["UID"],how = "left"  )
# mac_merch = mac_merch.fillna( 0.0 )

In [53]:
# len(mac_merch.columns)

In [104]:
# # for embedding mac1 and merchants pca
# from sklearn.decomposition import PCA
# dim = 42
# pca = PCA(n_components=dim)
# pca_res = pca.fit_transform(mac_merch.drop("UID", axis = 1)  )
# me_pca = pd.DataFrame(pca_res, columns=["pca_mac1_merchrant_%d" % i for i in range(dim)])
# me_pca["UID"] = mac_merch.UID.values

In [57]:
# datapca.head()

In [105]:
# datapca = pd.merge( data,me_pca,on = ["UID"],how = "left"  )
# data = pd.merge( data, mac1_emb, on =["UID"], how="left" )
# data = pd.merge( data, merchant_dbk, on =["UID"], how="left" )

In [32]:
import os
import pandas as pd
import numpy as np
import gc
import time
from contextlib import contextmanager
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import roc_auc_score, roc_curve, f1_score
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pickle
from sklearn.svm import SVC
def  process_feature(train_x, valid_x, test_df):
    result = []
    drop_cols = ['Tag']
    for df in [train_x, valid_x, test_df]:
        result.append(df.drop(drop_cols, axis=1))
    return result 
def cv(df, num_folds, param, stratified=True, debug=False):
    train_df =  df[df.Tag != 0.5]
    test_df =   df[df.Tag == 0.5]
    
    seed = 178
    if "seed" in param: 
        seed = param["seed"]
        del param['seed']
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=seed)
    oof_preds = np.zeros(train_df.shape[0])
    all_test_preds = []    
    feature_importance_df = pd.DataFrame()
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df, train_df['Tag'])):
        train_x, train_y = train_df.iloc[train_idx], train_df['Tag'].iloc[train_idx]
        valid_x, valid_y = train_df.iloc[valid_idx], train_df['Tag'].iloc[valid_idx]
        fold_preds = test_df[["UID"]]
        
        train_x, valid_x, test = process_feature(train_x, valid_x, test_df)
        if n_fold == 0:
            print(train_x.shape, valid_x.shape, test.shape)
        
        train_data = lgb.Dataset(train_x, label=train_y)
        validation_data = lgb.Dataset(valid_x, label=valid_y)

        clf=lgb.train(param,
                      train_data,
                      num_boost_round=10000,
                      valid_sets=[train_data, validation_data],
                      valid_names=["train", "valid"],
                      early_stopping_rounds=100,
                      verbose_eval=100)

        valid_preds = clf.predict(valid_x, num_iteration=clf.best_iteration)
        test_preds = clf.predict(test, num_iteration=clf.best_iteration)

        fold_preds['Tag'] = test_preds
        fold_preds['fold_id'] = n_fold + 1
        all_test_preds.append(fold_preds)
        
        oof_preds[valid_idx] = valid_preds
        
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, valid_preds)))
        
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()
    train_df["prob"] = oof_preds
    full_auc = roc_auc_score(train_df['Tag'], oof_preds)
    train_df = train_df[['UID', 'prob']]
    all_test_preds = pd.concat(all_test_preds, axis=0)
    test_preds = all_test_preds.groupby("UID")["Tag"].agg({"Tag":"mean"}).reset_index()
    del all_test_preds
    print('Full AUC score %.6f' % full_auc)
    
    return [train_df,test_preds]

In [33]:
class runmodel(object):
    def __init__(self,data):
        self.data = data
    def getinfo(self):
        print("use variable = runlgb.get_result() get reusult ")
    def get_embeding(self,fname, embname):
        f = open(fname)
        embeding_lines = f.readlines()
        f.close()
        mapfunc = lambda x: list( map( float, x ) )
        embeding_lines = [li.replace("\n","").split(" ") for li in embeding_lines[1:]]
        embeding_lines = [ [ int(line[0]) ] +  mapfunc( line[1:]  )   for line in   embeding_lines ]
        cols = ["UID"] + [ embname  + str(i) for i in range( len(embeding_lines[0]) -1 )]
        embeding_df = pd.DataFrame(embeding_lines, columns=cols )
        del embeding_lines
        return embeding_df
    def merge_emb2grdata(self,embdf):
        return pd.merge( self.data,embdf,on = ["UID"],how = "left"  )
    def get_result(self,fname, embname,subname):
        embdata = self.get_embeding(fname, embname)
        usedata = self.merge_emb2grdata(embdata)
        params_list =[
         {'boosting_type': 'goss', 'colsample_bytree': 0.6555004575000242, 'learning_rate': 0.016380004820033073, 'max_bin': 1000, 'metric': 'auc', 'min_child_weight': 2.1115838168176433, 'num_leaves': 108, 'reg_alpha': 23.247001339889128, 'reg_lambda': 997.9576062039534, 'subsample': 1.0, 'verbose': 1, 'seed': 4419},
         {'boosting_type': 'goss', 'colsample_bytree': 0.7187703092392053, 'learning_rate': 0.01939219215282862, 'max_bin': 310, 'metric': 'auc', 'min_child_weight': 2.66983907940641, 'num_leaves': 89, 'reg_alpha': 18.48224434106526, 'reg_lambda': 470.54675380054465, 'subsample': 1.0, 'verbose': 1, 'seed': 57},
         {'boosting_type': 'goss', 'colsample_bytree': 0.6274617979582582, 'learning_rate': 0.01680918441243103, 'max_bin': 780, 'metric': 'auc', 'min_child_weight': 0.8226071606806127, 'num_leaves': 73, 'reg_alpha': 14.466924422050258, 'reg_lambda': 658.5772060624658, 'subsample': 1.0, 'verbose': 1, 'seed': 1732},
         {'boosting_type': 'goss', 'colsample_bytree': 0.6991838451153098, 'learning_rate': 0.01577419276366034, 'max_bin': 940, 'metric': 'auc', 'min_child_weight': 7.758954855388241, 'num_leaves': 187, 'reg_alpha': 43.60868666926589, 'reg_lambda': 667.6371302027073, 'subsample': 1.0, 'verbose': 1, 'seed': 139},
         {'boosting_type': 'goss', 'colsample_bytree': 0.6198406436401038, 'learning_rate': 0.016662891953242748, 'max_bin': 820, 'metric': 'auc', 'min_child_weight': 3.5602833459924015, 'num_leaves': 77, 'reg_alpha': 13.398041512170746, 'reg_lambda': 631.8105595021391, 'subsample': 1.0, 'verbose': 1, 'seed': 7609}
        ]
        result = []
        for i, params in enumerate(params_list):
            model_dir = "./model_output/random/lgb_693/%d/" % i
            train_df,test_preds = cv(usedata , 5, params)
            train_pred_save = subname + '_train_%d_.csv'%i
            test_pred_save = subname + '_test_%d_.csv'%i
            train_df.to_csv(train_pred_save, encoding = 'utf8', index = False)
            test_preds.to_csv(test_pred_save, encoding = 'utf8', index = False)




In [35]:
import datetime
time_now = datetime.datetime.now()
crt_folder = time_now.strftime('%Y-%m-%d_') + str(time_now.toordinal())
runlgb = runmodel(data)
stackingRoot = cacheRoot + crt_folder
if not os.path.exists(stackingRoot):
    os.mkdir(stackingRoot)
    print("making sub root for stacking :%s"%stackingRoot )
    for embfN in [c for c in os.listdir(cacheRoot) if ".emb" in c]:
        embname = embfN.split("edglist")[0]
        subname = stackingRoot + "/"+ embname + "_embtype_" +  embfN.split(".")[-1]
        runlgb.get_result(cacheRoot + "/" + embfN, embname,subname)

        
    
    
# runlgb = runmodel(data)
# runlgb.getinfo()
# sub_embeding = runlgb.get_result()

# all_subs = [x[1] for x in sub_embeding]
# all_sub3 = pd.concat( all_subs  )
# all_sub3 = all_sub3.groupby("UID")["Tag"].agg({"Tag":"mean" }).reset_index()
# all_sub3.Tag = all_sub3.Tag * 2.5
# all_sub3.to_csv(subRoot + "sub2.csv", encoding = 'utf8', index = False)

making sub root for stacking :../cache/2018-12-16_737044
(24943, 503) (6236, 503) (31588, 503)
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.960953	valid's auc: 0.943097
[200]	train's auc: 0.975819	valid's auc: 0.957462
[300]	train's auc: 0.98164	valid's auc: 0.961531
[400]	train's auc: 0.985667	valid's auc: 0.963404
[500]	train's auc: 0.988591	valid's auc: 0.964632
[600]	train's auc: 0.990986	valid's auc: 0.965195
[700]	train's auc: 0.992749	valid's auc: 0.965576
[800]	train's auc: 0.994085	valid's auc: 0.965564
Early stopping, best iteration is:
[727]	train's auc: 0.993158	valid's auc: 0.965661


D:\Programfiles2\anaconda\lib\site-packages\ipykernel\__main__.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Programfiles2\anaconda\lib\site-packages\ipykernel\__main__.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Fold  1 AUC : 0.965661
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.92507	valid's auc: 0.917656
Early stopping, best iteration is:
[61]	train's auc: 0.953941	valid's auc: 0.940748


D:\Programfiles2\anaconda\lib\site-packages\ipykernel\__main__.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Programfiles2\anaconda\lib\site-packages\ipykernel\__main__.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Fold  2 AUC : 0.940748
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.963421	valid's auc: 0.944344
[200]	train's auc: 0.976518	valid's auc: 0.957771
[300]	train's auc: 0.982151	valid's auc: 0.961972
[400]	train's auc: 0.986215	valid's auc: 0.964039
[500]	train's auc: 0.989105	valid's auc: 0.965281
[600]	train's auc: 0.991349	valid's auc: 0.965957
[700]	train's auc: 0.993019	valid's auc: 0.966507
[800]	train's auc: 0.994287	valid's auc: 0.967286
[900]	train's auc: 0.995275	valid's auc: 0.967486
[1000]	train's auc: 0.996069	valid's auc: 0.967732
[1100]	train's auc: 0.996657	valid's auc: 0.9679
[1200]	train's auc: 0.997121	valid's auc: 0.967724
Early stopping, best iteration is:
[1142]	train's auc: 0.996849	valid's auc: 0.967978


...

Fold  4 AUC : 0.941324
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.970609	valid's auc: 0.954868
[200]	train's auc: 0.982125	valid's auc: 0.963257
[300]	train's auc: 0.987826	valid's auc: 0.965209
[400]	train's auc: 0.991476	valid's auc: 0.966109
[500]	train's auc: 0.993913	valid's auc: 0.966444
[600]	train's auc: 0.995585	valid's auc: 0.966908
[700]	train's auc: 0.996729	valid's auc: 0.96701
[800]	train's auc: 0.997527	valid's auc: 0.966866
Early stopping, best iteration is:
[712]	train's auc: 0.996859	valid's auc: 0.967122


D:\Programfiles2\anaconda\lib\site-packages\ipykernel\__main__.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Programfiles2\anaconda\lib\site-packages\ipykernel\__main__.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Fold  5 AUC : 0.967122


D:\Programfiles2\anaconda\lib\site-packages\ipykernel\__main__.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Full AUC score 0.961625


In [37]:
dataroot = "../data/"
cacheRoot = "../cache/"
subRoot = "../submission/"

pred_trn_dfs = []
pred_tsn_dfs = []
base_predictions_train_dic = {}
for embfN in [c for c in os.listdir(cacheRoot) if ".emb" in c]:
    embname = embfN.split("edglist")[0]
    subname = stackingRoot + "/"+ embname + "_embtype_" +  embfN.split(".")[-1]
    for i in range(5):
        train_pred_save = subname + '_train_%d_.csv'%i
        test_pred_save = subname + '_test_%d_.csv'%i
        pred_trn =  pd.read_csv(train_pred_save)["prob"].values.reshape(-1,1)
        base_predictions_train_dic[embname + "_"+  str(i) ] = pred_trn.ravel()
        pred_trn_dfs.append( pred_trn  )  
        pred_tsn_dfs.append( pd.read_csv(test_pred_save)["Tag"].values.reshape(-1,1)  )  

base_predictions_train = pd.DataFrame(base_predictions_train_dic)
base_predictions_train.head()


,device1__0,device1__1,device1__2,device1__3,device1__4,device_code2__0,device_code2__1,device_code2__2,device_code2__3,device_code2__4,...,merchant_weighted__0,merchant_weighted__1,merchant_weighted__2,merchant_weighted__3,merchant_weighted__4,wifi__0,wifi__1,wifi__2,wifi__3,wifi__4
0,0.146497,0.162197,0.169074,0.114448,0.180234,0.141879,0.168333,0.164825,0.108391,0.181037,...,0.128964,0.132806,0.121131,0.135579,0.131889,0.146788,0.167300,0.175654,0.109115,0.183270
1,-0.016352,-0.035719,-0.019262,-0.020743,-0.012715,-0.035957,-0.037478,-0.024742,-0.019539,-0.029202,...,-0.020451,-0.018022,-0.015941,-0.023442,-0.010101,-0.027618,-0.024981,-0.029128,-0.039541,-0.021486
2,0.101992,0.070607,0.103653,0.079822,0.118613,0.087591,0.062718,0.102726,0.076901,0.111804,...,0.077843,0.033912,0.112221,0.106386,0.031534,0.114063,0.067761,0.101370,0.079366,0.108927
3,-0.069394,-0.045161,-0.036481,-0.051028,-0.053031,-0.039171,-0.041032,-0.039254,-0.058202,-0.042554,...,-0.006597,0.013139,-0.009023,-0.001667,0.006729,-0.056490,-0.038447,-0.054578,-0.052559,-0.034356
4,-0.038110,0.000009,-0.019217,-0.032002,0.003202,-0.034347,-0.020580,-0.008864,-0.031328,-0.002515,...,-0.002245,0.007191,-0.010708,0.011915,-0.007378,-0.023684,-0.004185,-0.000328,-0.048756,0.003131


In [42]:
[c for c in os.listdir(cacheRoot) if ".emb" in c]

['device1_edglist_emb_dim36.emb',
 'device_code2_edglist_emb_dim36.emb',
 'geo_code_edglist.embeddings',
 'geo_code_edglist_emb_dim36.emb',
 'ip1_edglist.embeddings',
 'ip1_edglist_emb_dim36.emb',
 'mac1_edglist_emb_dim36.emb',
 'mac1_edglist_noweit_.embeddings',
 'merchant_edglist_emb_dim36.emb',
 'merchant_weighted_edglist_DeepWalk.embeddings',
 'wifi_edglist.embeddings',
 'wifi_edglist_emb_dim36.emb']

In [40]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
base_predictions_train_folder3 = base_predictions_train[[ c for c in base_predictions_train.columns if "_2" in c] ]
dataplt = [
    go.Heatmap(
        z= base_predictions_train_folder3.astype(float).corr().values ,
        x=base_predictions_train_folder3.columns.values,
        y= base_predictions_train_folder3.columns.values,
          colorscale='Viridis',
            showscale=True,
            reversescale = True
    )
]
py.iplot(dataplt, filename='labelled-heatmap')

In [41]:
from sklearn.cluster import KMeans

In [44]:
mec = runlgb.get_embeding("../cache/merchant_weighted_edglist_DeepWalk.embeddings","merchant_depwk")

In [49]:
kmdata = pd.DataFrame()
kmdata['UID'] =data.UID
kmdata['Tag'] =data.Tag
kmdata = kmdata.merge(mec )

In [85]:
y_pred = KMeans(n_clusters=10, random_state=66).fit_predict( kmdata[[c for c in kmdata.columns if c not in [ "UID","Tag"] ]]  )
kmdata["Cluster"] = y_pred
kmdata[kmdata.Tag != 0.5 ][['Cluster','Tag' ]].corr()

,Cluster,Tag
Cluster,1.000000,-0.186966
Tag,-0.186966,1.000000


,Cluster,Tag
Cluster,1.000000,0.210431
Tag,0.210431,1.000000


,Cluster,Tag
Cluster,1.000000,0.173485
Tag,0.173485,1.000000


In [ ]:
x_train = np.concatenate( pred_trn_dfs ,axis=1) 
x_test  = np.concatenate( pred_tsn_dfs ,axis=1) 
y_train = pd.read_csv("../data/tag_train_new.csv").Tag.values.reshape(-1,1)

In [36]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier

def fpreproc(dtrain, dtest, param):
    label = dtrain.get_label()
    ratio = float(np.sum(label == 0)) / np.sum(label == 1)
    param['scale_pos_weight'] = ratio
    return (dtrain, dtest, param)

gbm = xgb.XGBClassifier(
    #learning_rate = 0.02,
 n_estimators= 2000,
 max_depth= 4,
 min_child_weight= 2,
 #gamma=1,
 gamma=0.9,                        
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread= -1,
 fpreproc=fpreproc).fit(x_train, y_train)
predictions = gbm.predict(x_test)

NameError: name 'x_train' is not defined